### Question-1 , A. DCGAN Training on CIFAR-10

In [ ]:
pip install torch torchvision matplotlib


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [ ]:
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 3*32*32),
            nn.Tanh()
        )

    def forward(self, z):
        return self.fc(z).view(-1, 3, 32, 32)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(3*32*32, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x.view(-1, 3*32*32))


In [ ]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()


In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# To track the losses
d_losses, g_losses = [], []

num_epochs = 50
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(train_loader):
        batch_size = real_images.size(0)
        real_images = real_images.cuda()

        # Real labels are 1, fake labels are 0
        real_labels = torch.ones(batch_size, 1).cuda()
        fake_labels = torch.zeros(batch_size, 1).cuda()

        # Train Discriminator
        optimizer_d.zero_grad()
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, 100).cuda()
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_g.step()

        # Save losses
        d_losses.append(d_loss_real.item() + d_loss_fake.item())
        g_losses.append(g_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss_real.item() + d_loss_fake.item():.4f}, g_loss: {g_loss.item():.4f}')


### B. Plot Generator and Discriminator Losses

In [ ]:
# Plot the losses
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Losses")
plt.plot(np.arange(len(d_losses)), d_losses, label='Discriminator Loss')
plt.plot(np.arange(len(g_losses)), g_losses, label='Generator Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Generate random noise
z = torch.randn(64, 100).cuda()

# Generate fake images
fake_images = generator(z)

# Convert images back to CPU and visualize
fake_images = fake_images.cpu().detach()
grid = torchvision.utils.make_grid(fake_images, nrow=8, padding=2, normalize=True)
plt.imshow(grid.permute(1, 2, 0))
plt.axis('off')
plt.show()


### Question-2, A. The complete network is trained from scratch (i.e, random weights)  

### B. A pre-trained ResNet50 on ImageNet weights is used and only the neural network layers are trained    

### C. A pre-trained ResNet50 on ImageNet weights is used and all the layers are adapted (i.

### D. Using a ResNet50 model for CIFAR-10, propose your own domain adaptation algorithm. 

### Question 3: Implement a gan from scratch using Keras to generate celebrity faces from noise using this celeba data

  Use cases found for GAN

<br> Super-resolution: increasing the resolution of input image%
<br> Colorise blank and white image%
<br> image inpainting - fill missing blocks in image%
<br> Anime face generation
<br> font generation
<br> style transfer
<br> human face generation
<br> image to emoj'
<br> GAN for data augmentation
<br> Face ageing GAN
<br> front facial view generation from images provided of different side%
<br> Photo blending- blending 2 images

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Load CelebA dataset from directory
# Assuming the images are stored in the 'celeba' directory with class folders for simplicity
datagen = ImageDataGenerator(rescale=1./255)

# Load dataset
data = datagen.flow_from_directory('clist_attr_celeba.csv', target_size=(64, 64), batch_size=128, class_mode=None)

# Normalize and store images
X_train = data.next()
X_train = X_train * 2 - 1  # Normalize to the range [-1, 1]


In [ ]:
from tensorflow.keras import layers, models

def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=100))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Reshape((16, 16, 1)))
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Conv2DTranspose(3, kernel_size=3, strides=2, padding='same', activation='tanh'))
    
    return model

generator = build_generator()
generator.summary()


In [ ]:
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=(64, 64, 3), padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

discriminator = build_discriminator()
discriminator.summary()


In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

gan = build_gan(generator, discriminator)
gan.summary()


In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))


In [ ]:
import os
from tensorflow.keras.preprocessing.image import save_img

def train_gan(generator, discriminator, gan, epochs, batch_size, sample_interval):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train discriminator with real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]
        real_labels = np.ones((half_batch, 1))
        
        # Train discriminator with fake images
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

        # Update discriminator
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator (via GAN)
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))  # Trick the generator
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress
        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")

        # Save generated image samples
        if epoch % sample_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    noise = np.random.normal(0, 1, (25, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0,1]
    
    fig, axs = plt.subplots(5, 5)
    cnt = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(generated_images[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_{epoch}.png")
    plt.close()

# Train the GAN
train_gan(generator, discriminator, gan, epochs=10000, batch_size=64, sample_interval=1000)
